In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "facebook/nllb-200-distilled-600M"
SAVE_PATH = "/content/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

tokenizer.save_pretrained(SAVE_PATH)
model.save_pretrained(SAVE_PATH)

print("Modelo guardado en:", SAVE_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Modelo guardado en: /content/nllb-200-distilled-600M


In [ ]:
# TRADUCCIÓN BLEU DATASET: ESPAÑOL → QUECHUA (NLLB-200 FINE-TUNING)

import csv
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# CONFIGURACIÓN
INPUT_CSV  = "/content/dataset_metricas.csv"
OUTPUT_CSV = "/content/dataset_metricas.csv_traducido.csv"

MODEL_NAME = "nllb-200-distilled-600M-FT"

SRC_LANG = "spa_Latn"
TGT_LANG = "quy_Latn"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# CARGA MODELO
print("Cargando modelo")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(DEVICE)

tokenizer.src_lang = SRC_LANG
FORCED_BOS = tokenizer.convert_tokens_to_ids(TGT_LANG)

assert FORCED_BOS > 10, "ERROR: token de idioma destino inválido"

print(f"Dispositivo: {DEVICE}")
print(f"Traducción: {SRC_LANG} → {TGT_LANG}")
print(f"Token destino ID: {FORCED_BOS}")

# FUNCIÓN DE TRADUCCIÓN

def traducir(texto: str) -> str:
    if not texto or not texto.strip():
        return ""

    inputs = tokenizer(
        texto,
        return_tensors="pt",
        truncation=True,
        max_length=384
    ).to(DEVICE)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=256,
            forced_bos_token_id=FORCED_BOS,
            num_beams=4,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )

    return tokenizer.decode(
        output[0],
        skip_special_tokens=True
    ).strip()

# PROCESAR CSV

filas_salida = []

with open(INPUT_CSV, "r", encoding="utf-8-sig") as f:
    reader = csv.reader(f, delimiter=";")

    for i, fila in enumerate(reader, 1):
        if not fila or not fila[0].strip():
            filas_salida.append(fila)
            continue

        texto_es = fila[0]
        print(f"Traduciendo fila {i}")

        traduccion_qu = traducir(texto_es)

        # Asegurar al menos 3 columnas
        while len(fila) < 3:
            fila.append("")

        fila[2] = traduccion_qu
        filas_salida.append(fila)

# GUARDAR CSV

with open(OUTPUT_CSV, "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(filas_salida)

print("===================================")
print("TRADUCCIÓN COMPLETADA")
print(f"Archivo generado: {OUTPUT_CSV}")
print(f"Filas traducidas: {len(filas_salida)}")


Cargando modelo NLLB-200
Dispositivo: cuda
Traducción: spa_Latn → quy_Latn
Token destino ID: 256144
Traduciendo fila 1
Traduciendo fila 2
Traduciendo fila 3
Traduciendo fila 4
Traduciendo fila 5
Traduciendo fila 6
Traduciendo fila 7
Traduciendo fila 8
Traduciendo fila 9
Traduciendo fila 10
Traduciendo fila 11
Traduciendo fila 12
Traduciendo fila 13
Traduciendo fila 14
Traduciendo fila 15
Traduciendo fila 16
Traduciendo fila 17
Traduciendo fila 18
Traduciendo fila 19
Traduciendo fila 20
Traduciendo fila 21
Traduciendo fila 22
Traduciendo fila 23
Traduciendo fila 24
Traduciendo fila 25
Traduciendo fila 26
Traduciendo fila 27
Traduciendo fila 28
Traduciendo fila 29
Traduciendo fila 30
Traduciendo fila 31
Traduciendo fila 32
Traduciendo fila 33
Traduciendo fila 34
Traduciendo fila 35
Traduciendo fila 36
Traduciendo fila 37
Traduciendo fila 38
Traduciendo fila 39
Traduciendo fila 40
Traduciendo fila 41
Traduciendo fila 42
Traduciendo fila 43
Traduciendo fila 44
Traduciendo fila 45
Traducien

In [ ]:
!pip install -q sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
# Cargar CSV y preparar datos

import csv

CSV_PATH = "/content/dataset_metricas.csv"

refs = []
hyps = []

with open(CSV_PATH, "r", encoding="utf-8-sig") as f:
    reader = csv.reader(f, delimiter=";")
    next(reader, None)  # saltar header

    for row in reader:
        if len(row) < 2:
            continue

        src = row[0].strip()   # español (no se usa para métricas)
        ref = row[1].strip()   # quechua (referencia)

        # EJEMPLO: si aún no tienes hipótesis del modelo,
        # usa la referencia como placeholder (BLEU=100)
        hyp = ref

        if ref and hyp:
            refs.append(ref)
            hyps.append(hyp)

print(f"Pares válidos cargados: {len(hyps)}")
print("Ejemplo REF:", refs[0])
print("Ejemplo HYP:", hyps[0])


Pares válidos cargados: 1812
Ejemplo REF: Manan noqaqa rikunichu
Ejemplo HYP: Manan noqaqa rikunichu


In [ ]:
!pip install -q transformers accelerate bitsandbytes sentencepiece
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 146.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 22.1 MB/s eta 0:00:00


In [ ]:
!pip install -q -U bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 171.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 153.1 MB/s eta 0:00:00


In [ ]:
# Calculo de BLUE y CHRf

import sacrebleu

# sacreBLEU espera referencias como lista de listas
references = [refs]

bleu = sacrebleu.corpus_bleu(hyps, references)
chrf = sacrebleu.corpus_chrf(hyps, references)

print("===================================")
print(f"BLEU    : {bleu.score:.2f}")
print(f"chrF++  : {chrf.score:.2f}")
print("===================================")


BLEU    : 100.00
chrF++  : 100.00


In [ ]:
import csv
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

DEVICE: cuda


In [ ]:
NLLB_NAME = "/content/nllb-200-distilled-600M-FT"

tok_nllb = AutoTokenizer.from_pretrained(NLLB_NAME)
mod_nllb = AutoModelForSeq2SeqLM.from_pretrained(NLLB_NAME).to(DEVICE)

tok_nllb.src_lang = "spa_Latn"
BOS_QU = tok_nllb.convert_tokens_to_ids("quy_Latn")


In [ ]:
def traducir_nllb(texto):
    if not texto.strip():
        return ""
    inp = tok_nllb(texto, return_tensors="pt", truncation=True, max_length=384).to(DEVICE)
    out = mod_nllb.generate(
        **inp,
        forced_bos_token_id=BOS_QU,
        num_beams=4,
        max_new_tokens=256
    )
    return tok_nllb.decode(out[0], skip_special_tokens=True)


In [ ]:
!pip install -U bitsandbytes accelerate

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig
)

# Dispositivo
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Modelo
MADLAD_NAME = "google/madlad400-3b-mt"

# Configuración correcta de cuantización 8-bit
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True
)
# Tokenizer
tok_madlad = AutoTokenizer.from_pretrained(MADLAD_NAME)
# Modelo (FORMA CORRECTA)
mod_madlad = AutoModelForSeq2SeqLM.from_pretrained(
    MADLAD_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

def traducir_madlad(texto):
    if not texto.strip():
        return ""
    prompt = f"<2qu> {texto}"
    inp = tok_madlad(prompt, return_tensors="pt").to(mod_madlad.device)
    out = mod_madlad.generate(**inp, max_new_tokens=256)
    return tok_madlad.decode(out[0], skip_special_tokens=True)


In [ ]:
QWEN_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

tok_qwen = AutoTokenizer.from_pretrained(QWEN_NAME)
mod_qwen = AutoModelForCausalLM.from_pretrained(
    QWEN_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

def traducir_qwen(texto):
    if not texto.strip():
        return ""
    prompt = (
        "Traduce el siguiente texto del español al quechua (variante Cusco-Collao):\n"
        f"{texto}\n\nQuechua:"
    )
    inp = tok_qwen(prompt, return_tensors="pt").to(DEVICE)
    out = mod_qwen.generate(**inp, max_new_tokens=256)
    return tok_qwen.decode(out[0], skip_special_tokens=True).split("Quechua:")[-1].strip()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
INPUT_CSV  = "/content/dataset_metricas.csv"
OUTPUT_CSV = "/content/dataset_metricas_full.csv"

salida = []

with open(INPUT_CSV, "r", encoding="utf-8-sig") as f:
    reader = csv.reader(f, delimiter=";")

    for i, fila in enumerate(reader, 1):
        while len(fila) < 5:
            fila.append("")

        es = fila[0]

        print(f"Fila {i}")

        if not fila[2]:
            fila[2] = traducir_nllb(es)

        if not fila[3]:
            fila[3] = traducir_madlad(es)

        if not fila[4]:
            fila[4] = traducir_qwen(es)

        salida.append(fila)

with open(OUTPUT_CSV, "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(salida)

print("CSV FINAL:", OUTPUT_CSV)


Fila 1
Fila 2
Fila 3
Fila 4
Fila 5
Fila 6
Fila 7
Fila 8
Fila 9
Fila 10
Fila 11
Fila 12
Fila 13
Fila 14
Fila 15
Fila 16
Fila 17
Fila 18
Fila 19
Fila 20
Fila 21
Fila 22
Fila 23
Fila 24
Fila 25
Fila 26
Fila 27
Fila 28
Fila 29
Fila 30
Fila 31
Fila 32
Fila 33
Fila 34
Fila 35
Fila 36
Fila 37
Fila 38
Fila 39
Fila 40
Fila 41
Fila 42
Fila 43
Fila 44
Fila 45
Fila 46
Fila 47
Fila 48
Fila 49
Fila 50
Fila 51
Fila 52
Fila 53
Fila 54
Fila 55
Fila 56
Fila 57
Fila 58
Fila 59
Fila 60
Fila 61
Fila 62
Fila 63
Fila 64
Fila 65
Fila 66
Fila 67
Fila 68
Fila 69
Fila 70
Fila 71
Fila 72
Fila 73
Fila 74
Fila 75
Fila 76
Fila 77
Fila 78
Fila 79
Fila 80
Fila 81
Fila 82
Fila 83
Fila 84
Fila 85
Fila 86
Fila 87
Fila 88
Fila 89
Fila 90
Fila 91
Fila 92
Fila 93
Fila 94
Fila 95
Fila 96
Fila 97
Fila 98
Fila 99
Fila 100
Fila 101
Fila 102
Fila 103
Fila 104
Fila 105
Fila 106
Fila 107
Fila 108
Fila 109
Fila 110
Fila 111
Fila 112
Fila 113
Fila 114
Fila 115
Fila 116
Fila 117
Fila 118
Fila 119
Fila 120
Fila 121
Fila 122
Fila 123
F

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.3 MB/s eta 0:00:00


In [ ]:
# EVALUACIÓN BLEU / chrF POR MODELO

import csv
import sacrebleu

CSV_PATH = "/content/dataset_bleu_traducido_full.csv"  # <-- ajusta si es necesario

refs = []
nllb = []
madlad = []
qwen = []

# -------------------------
# CARGAR CSV (SIN HEADER)
# -------------------------
with open(CSV_PATH, "r", encoding="utf-8-sig") as f:
    reader = csv.reader(f, delimiter=";")

    for i, row in enumerate(reader, 1):
        if len(row) < 5:
            continue

        ref = row[1].strip()
        h_nllb = row[2].strip()
        h_madlad = row[3].strip()
        h_qwen = row[4].strip()

        if ref and h_nllb and h_madlad and h_qwen:
            refs.append(ref)
            nllb.append(h_nllb)
            madlad.append(h_madlad)
            qwen.append(h_qwen)

print(f"Pares evaluables: {len(refs)}")

references = [refs]

# -------------------------
# FUNCIÓN DE EVALUACIÓN
# -------------------------
def evaluar(nombre, hyps):
    bleu = sacrebleu.corpus_bleu(hyps, references)
    chrf = sacrebleu.corpus_chrf(hyps, references)

    print(f"\n🔹 {nombre}")
    print(f"BLEU   : {bleu.score:.2f}")
    print(f"chrF++ : {chrf.score:.2f}")

# -------------------------
# EVALUAR CADA MODELO
# -------------------------
evaluar("NLLB-200", nllb)
evaluar("MADLAD-400-3B", madlad)
evaluar("Qwen2.5-1.5B-Instruct", qwen)


Pares evaluables: 100

🔹 NLLB-200
BLEU   : 0.66
chrF++ : 27.56

🔹 MADLAD-400-3B
BLEU   : 0.18
chrF++ : 17.19

🔹 Qwen2.5-1.5B-Instruct
BLEU   : 0.01
chrF++ : 6.13


In [ ]:
!pip install -q unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 45.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 

In [ ]:
def evaluar_comet(nombre, hyps):
    data = [
        {
            "src": s,
            "mt": h,
            "ref": r
        }
        for s, h, r in zip(src, hyps, refs)
        if s and h and r
    ]

    scores = comet_model.predict(data, batch_size=8, gpus=1)
    score = sum(scores.scores) / len(scores.scores)

    print(f"\n🔹 {nombre}")
    print(f"COMET : {score:.4f}")


In [ ]:
!pip install -q unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 147.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 59.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version 

In [ ]:
# COMET POR MODELO (ESP -> QUECHUA)


import csv
from comet import download_model, load_from_checkpoint

# -------------------------
# CONFIG
# -------------------------
CSV_PATH = "/content/dataset_bleu_traducido_full.csv"

# -------------------------
# CARGAR MODELO COMET
# -------------------------
model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(model_path)

print("COMET cargado correctamente")

# -------------------------
# CARGAR DATA (SIN HEADER)
# -------------------------
src = []
refs = []
nllb = []
madlad = []
qwen = []

with open(CSV_PATH, "r", encoding="utf-8-sig") as f:
    reader = csv.reader(f, delimiter=";")
    for row in reader:
        if len(row) < 5:
            continue

        s = row[0].strip()
        r = row[1].strip()
        h1 = row[2].strip()
        h2 = row[3].strip()
        h3 = row[4].strip()

        if s and r and h1 and h2 and h3:
            src.append(s)
            refs.append(r)
            nllb.append(h1)
            madlad.append(h2)
            qwen.append(h3)

print(f"Ejemplos evaluables: {len(src)}")


# FUNCIÓN COMET

def evaluar_comet(nombre, hyps):
    data = [
        {"src": s, "mt": h, "ref": r}
        for s, h, r in zip(src, hyps, refs)
    ]

    outputs = comet_model.predict(
        data,
        batch_size=8,
        gpus=1 if comet_model.device.type == "cuda" else 0
    )

    score = sum(outputs.scores) / len(outputs.scores)

    print(f"\n🔹 {nombre}")
    print(f"COMET : {score:.4f}")


# EVALUAR MODELOS

evaluar_comet("NLLB-200", nllb)
evaluar_comet("MADLAD-400-3B", madlad)
evaluar_comet("Qwen2.5-1.5B-Instruct", qwen)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

LICENSE: 0.00B [00:00, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

COMET cargado correctamente


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset_bleu_traducido_full.csv'